In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle as pkl
from hierataxo.util import (
    OrderManager)
order_manager=OrderManager(pkl.load(open('taxo_data/hierarchy_order.pkl','rb'))['Riboviria'],
                        level_names=['Kingdom','Phylum','Class','Order'])

/home/taxo_playground/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
from hierataxo import (
    OrderManager,ConcatProteinDataset,
    HierarESM,HierarchicalLossNetwork,
    cal_accuracy,set_seed)
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset, DistributedSampler,random_split
device=0
batch_size=2
batch_size_val=25
max_domain=15
acc_step=20
valid_step=1000
max_length=500
to_freeze=3
hierar_esmmodel=HierarESM(order_manager=order_manager,max_length=max_length,to_freeze=to_freeze)
hierar_loss=HierarchicalLossNetwork(order_manager)
optimizer = Adam(hierar_esmmodel.parameters(), lr=1e-4)





/home/taxo_playground/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/taxo_playground/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by de

In [117]:
dataset=ConcatProteinDataset('taxo_data/proseq_taxo_1.pkl',order_manager,max_length=max_length)
trainset,valset=random_split(dataset,[0.9,0.1])
train_generator = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=batch_size,collate_fn=dataset.collate_fn)
val_generator = DataLoader(valset, batch_size=batch_size_val, shuffle=True, num_workers=batch_size,collate_fn=dataset.collate_fn)

/home/taxo_playground/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [60]:
dataset.data.columns

Index(['seq', 'taxo', 'family'], dtype='object')

In [37]:
dataset[2]
ipt = [hierar_esmmodel.tokenizer(dataset[2]['seq'], return_tensors="pt",padding='max_length',truncation=True,max_length=hierar_esmmodel.max_length)]


In [46]:
dataset[2].keys()

dict_keys(['name', 'seq', 'taxo_str', 'taxo', 'sentence_mask', 'input_ids', 'attention_mask'])

In [67]:
dataset[2]['sentence_mask']

tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [58]:
import torch
torch.unsqueeze(torch.tensor([0,1,2]),0).shape

torch.Size([1, 3])

In [79]:
_tmp=dataset[2]['sentence_mask']
torch.stack((_tmp,_tmp),dim=0).shape

torch.Size([2, 15])

In [125]:
for i in val_generator:
    break
# _=hierar_esmmodel.process_sample(i)

In [127]:
i.keys()

dict_keys(['name', 'seq', 'taxo_label', 'domain_label', 'taxo', 'sentence_mask', 'input_ids', 'attention_mask'])

In [100]:
ipt={'input_ids':i['input_ids'].view(-1,hierar_esmmodel.max_length),
                'attention_mask':i['attention_mask'].view(-1,hierar_esmmodel.max_length)
                }

In [120]:
dataset.max_length

500

In [118]:
ipt['input_ids'].shape

torch.Size([450, 500])

In [101]:
sentence_mask=i['sentence_mask'].view(-1)

In [110]:
i['input_ids'].shape

torch.Size([25, 15, 600])

In [108]:
25*15

375

In [102]:
sentence_mask.shape

torch.Size([375])

In [121]:
hierar_esmmodel(**i)

[tensor([[-0.1011, -0.3040, -0.1770],
         [-0.1215,  0.0918,  0.0823],
         [-0.0832, -0.0580, -0.0744],
         [-0.0710,  0.1848, -0.0478],
         [-0.0338,  0.0688, -0.1325],
         [-0.1790, -0.0649, -0.0780],
         [-0.2403,  0.1517, -0.1249],
         [-0.1133, -0.0387, -0.2101],
         [-0.0094,  0.0968, -0.0202],
         [ 0.0976,  0.1289, -0.1876],
         [-0.0766,  0.0429, -0.1378],
         [-0.0632, -0.0496, -0.2000],
         [-0.1372, -0.0260, -0.2431],
         [-0.0637, -0.1707,  0.0834],
         [-0.0007,  0.0469, -0.1799],
         [ 0.1343,  0.2033,  0.1073],
         [-0.1509,  0.0792, -0.1511],
         [-0.2284, -0.1360,  0.0724],
         [-0.3278,  0.0401, -0.2886],
         [-0.1276,  0.1229, -0.1437],
         [-0.0673, -0.1207, -0.1801],
         [-0.0927,  0.0143, -0.0757],
         [-0.0591, -0.1956, -0.2955],
         [ 0.0368,  0.1755,  0.0530],
         [-0.1123,  0.0627,  0.0229]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.1435,  0.

In [89]:
i.keys()

dict_keys(['name', 'seq', 'taxo_label', 'domain_label', 'taxo', 'sentence_mask', 'input_ids', 'attention_mask'])

In [126]:
i['taxo']

tensor([[ 1,  2,  4,  6],
        [ 1,  3, 10, 12],
        [ 1,  1,  2,  2],
        [ 1,  1,  2,  2],
        [ 1,  4, 16, 20],
        [ 1,  2,  4,  5],
        [ 1,  4, 16, 20],
        [ 1,  3, 10, 13],
        [ 1,  3, 10, 13],
        [ 1,  2,  4,  6],
        [ 1,  2,  4,  6],
        [ 1,  4, 14, 18],
        [ 1,  4, 14, 18],
        [ 1,  4, 16, 20],
        [ 1,  4, 16, 20],
        [ 1,  3, 10, 13],
        [ 1,  4, 16, 20],
        [ 1,  3, 11, 14],
        [ 1,  4, 16, 20],
        [ 1,  1,  2,  2],
        [ 1,  3, 10, 12],
        [ 1,  4, 16, 20],
        [ 2,  6, 21, 30],
        [ 1,  5, 19, 24],
        [ 1,  4, 14, 17]])

In [94]:
i['domain_label']

[['BDV_G', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['BDV_G', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['Bunya_NS-S_2', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['Phage_mat-A',
  'RNA_replicase_B',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['Mononeg_RNA_pol',
  'Mononeg_mRNAcap',
  'Methyltrans_Mon',
  'BDV_G',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['Sigma_1_2', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['B-CoV_A_NSP1',
  'bCoV_NSP3_N',
  'bCoV_SUD_M',
  'bCoV_SUD_M',
  'bCoV_SUD_C',
  'bCoV_NAB',
  'CoV_NSP3_C',
  'CoV_NSP4_N',
  'CoV_NSP6',
  'CoV_NSP6',
  'CoV_NSP10',
  'RdRP_3',
  'DUF2075',
  'CoV_nucleocap',
  'CoV_nucleocap'],
 ['Paramyxo_ncap',
  'Matrix',
  'Fusion_gly',
  'HN',
  'Mononeg_RNA_pol',
  'Mononeg_mRNAcap',
  'Methyltrans_Mon',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['Peptidase_S30',
  'Peptidase_C6',
  'Potyvirid-P3',
 

In [74]:
i['sentence_mask'][2]

tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [69]:
len(i['seq'][0])

25

In [20]:
len(i['seq'])

15

In [4]:
torch.pow(1.3,torch.tensor([0,1,2]))

tensor([1.0000, 1.3000, 1.6900])

In [23]:
?torch.nn.functional.one_hot

Docstring:
one_hot(tensor, num_classes=-1) -> LongTensor

Takes LongTensor with index values of shape ``(*)`` and returns a tensor
of shape ``(*, num_classes)`` that have zeros everywhere except where the
index of last dimension matches the corresponding value of the input tensor,
in which case it will be 1.

See also `One-hot on Wikipedia`_ .

.. _One-hot on Wikipedia:
    https://en.wikipedia.org/wiki/One-hot

Arguments:
    tensor (LongTensor): class values of any shape.
    num_classes (int):  Total number of classes. If set to -1, the number
        of classes will be inferred as one greater than the largest class
        value in the input tensor.

Returns:
    LongTensor that has one more dimension with 1 values at the
    index of last dimension indicated by the input, and 0 everywhere
    else.

Examples:
    >>> F.one_hot(torch.arange(0, 5) % 3)
    tensor([[1, 0, 0],
            [0, 1, 0],
            [0, 0, 1],
            [1, 0, 0],
            [0, 1, 0]])
    >>> F.one_ho

In [7]:

tax=lambda x:1.13*x
shared=[15.99,
        12.99,
        35.45]
exclusive=[7.49,
           tax(18.99-4.5),
           tax(19.99-5),
           tax(9.99),
           11.99,
           8.99,
           8.99]

sum(shared)/2+sum(exclusive)

114.2761

In [10]:
order_manager.levels

[['Null', 'Orthornavirae', 'Pararnavirae'],
 ['Null',
  'Duplornaviricota',
  'Kitrinoviricota',
  'Lenarviricota',
  'Negarnaviricota',
  'Pisuviricota',
  'Artverviricota'],
 ['Null',
  'Chrymotiviricetes',
  'Resentoviricetes',
  'Vidaverviricetes',
  'Alsuviricetes',
  'Flasuviricetes',
  'Magsaviricetes',
  'Tolucaviricetes',
  'Amabiliviricetes',
  'Howeltoviricetes',
  'Leviviricetes',
  'Miaviricetes',
  'Chunqiuviricetes',
  'Milneviricetes',
  'Monjiviricetes',
  'Yunchangviricetes',
  'Ellioviricetes',
  'Insthoviricetes',
  'Duplopiviricetes',
  'Pisoniviricetes',
  'Stelpaviricetes',
  'Revtraviricetes'],
 ['Null',
  'Ghabrivirales',
  'Reovirales',
  'Mindivirales',
  'Hepelivirales',
  'Martellivirales',
  'Tymovirales',
  'Amarillovirales',
  'Nodamuvirales',
  'Tolivirales',
  'Wolframvirales',
  'Cryppavirales',
  'Norzivirales',
  'Timlovirales',
  'Ourlivirales',
  'Muvirales',
  'Serpentovirales',
  'Jingchuvirales',
  'Mononegavirales',
  'Goujianvirales',
  'Buny